这是一个分类问题，所以最好使用交叉熵损失，隐藏层使用 ReLU 激活函数，输出层使用 softmax 函数

In [1]:
import tensorflow as tf
import tensorflow.contrib.layers as layers
from tensorflow.python import debug as tf_debug

神经网络参数

In [2]:
n_hidden = 30
n_classes = 10
n_input = 784

超参数

In [3]:
batch_size = 200
eta = 0.001
max_epoth = 10

MINST输入数据

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
def multilayer_perceptron(x):
    fc1 = layers.fully_connected(x,n_hidden,activation_fn=tf.nn.relu,scope='fc1')
    out = layers.fully_connected(fc1,n_classes,activation_fn=None,scope='out')
    return out

建立模型，损失函数，和op

In [8]:
x = tf.placeholder(tf.float32,[None,n_input],name='placeholder_x')
y = tf.placeholder(tf.float32,[None,n_classes],name='placeholder_y')
y_hat = multilayer_perceptron(x)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hat,labels=y))
train = tf.train.AdamOptimizer(learning_rate=eta).minimize(loss)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        epoch_loss = 0.0
        batch_steps = int(mnist.train.num_examples / batch_size)
        for i in range(batch_steps):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            _,c = sess.run([train,loss],feed_dict={x:batch_x,y:batch_y})
            epoch_loss += c / batch_steps
        print('Epoch %02d,Loss = %.6f' % (epoch,epoch_loss))
     
    # 测试
    correct_prediction = tf.equal(tf.arg_max(y_hat,1),tf.arg_max(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print("Accuracy%:",accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 00,Loss = 0.631747
Epoch 01,Loss = 0.291902
Epoch 02,Loss = 0.242122
Epoch 03,Loss = 0.210166
Epoch 04,Loss = 0.188151
Epoch 05,Loss = 0.170226
Epoch 06,Loss = 0.156294
Epoch 07,Loss = 0.144516
Epoch 08,Loss = 0.134122
Epoch 09,Loss = 0.125039
Instructions for updating:
Use `tf.math.argmax` instead
Accuracy%: 0.9597


改后的 MNIST MLP 分类器在测试数据集上只用了一个隐藏层，并且在 10 个 epoch 内，只需要几行代码，就可以得到 96% 的精度：